<a href="https://colab.research.google.com/github/Shiladitya56/Machine-Learning/blob/main/SEM3/Natural%20Language%20Processing/lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This is a snippet of my product ChatExcel which is the future of talking to your excel files.

#Ask me for demo 😀!

## shiladitya©


In [ ]:
#created by shiladitya©

import os
from dotenv import load_dotenv
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.chains import create_sql_query_chain
import pandas as pd
from sqlalchemy import create_engine

class Shila:
    def __init__(shila, df):
        df = pd.read_excel(df, engine='openpyxl')
        engine = create_engine('sqlite:///DEMO.db', echo=False)
        load_dotenv('C:\StrangerCodes\ChatExcel\secret.env')
        shila.llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=os.environ['GK'])
        df.to_sql('DEMO', con=engine, if_exists='replace', index=False)

    def say(shila, q):

        db = SQLDatabase.from_uri('sqlite:///DEMO.db')

        template = '''You are an SQLite expert. Given an input question, create a syntactically correct {dialect} query to run.
        Pay attention to use only the column names you can see in the table below. Be careful to not query for columns that do not exist.
        The top_k value is {top_k}.

        Use the following format:

        Question: "Question here"
        Answer: "Final answer here"

        In the Answer, DO NOT write anything (INCLUDING the word Answer:) but just the SQL query.
        For example -
        Question: "Get all the records."
        Answer: "SELECT COUNT(*) FROM DEMO;"

        Only use the following table:

        {table_info}.

        Question: {input}'''

        prompt = PromptTemplate.from_template(template)

        execute_query = QuerySQLDataBaseTool(db=db)
        write_query = create_sql_query_chain(shila.llm, db, prompt=prompt)
        chain = write_query | execute_query
        response = chain.invoke(dict(question=q, top_k = 50, table_info=db.get_table_info(), dialect=db.dialect))

        ftemp = f'''
            I will give you information about an SQL table, a query and the records generated by that query.
            You will see the records that are generated and write the same in natural language.
            For example:
            Table Information: Name: Alice, Bob, ...
                               Age: 34, 26, ...
            Query: How many records are there?
            Records Generated: 8
            Your answer: There are a total of 8 records in the table.

            Now fill Your answer in this -
            Table Information: {db.table_info}
            Query: {q}
            Records Generated: {response}
            Your answer:
        '''

        return shila.llm.invoke(ftemp).content
